In [6]:
import gurobipy as gp

In [7]:
# Dados brutos
qtd_itens = 20
qtd_caixas = 8 # Limitante superior obtido pelo algoritmo Next Fit
capacidade = 400
lista_pesos = [53, 89, 221, 61, 73, 241, 153, 167, 194, 127, 193, 191, 79, 169, 97, 55, 225, 142, 150, 90]

print (lista_pesos)

[53, 89, 221, 61, 73, 241, 153, 167, 194, 127, 193, 191, 79, 169, 97, 55, 225, 142, 150, 90]


In [8]:
# Lista com os Indices dos itens
itens = ["Item_{}".format(i + 1) for i in range(qtd_itens)]
print(itens)
caixas = ["Caixa_{}".format(i + 1) for i in range(qtd_caixas)]
print(caixas)

['Item_1', 'Item_2', 'Item_3', 'Item_4', 'Item_5', 'Item_6', 'Item_7', 'Item_8', 'Item_9', 'Item_10', 'Item_11', 'Item_12', 'Item_13', 'Item_14', 'Item_15', 'Item_16', 'Item_17', 'Item_18', 'Item_19', 'Item_20']
['Caixa_1', 'Caixa_2', 'Caixa_3', 'Caixa_4', 'Caixa_5', 'Caixa_6', 'Caixa_7', 'Caixa_8']


In [9]:
pesos = dict()
for i in range(qtd_itens):
    pesos[itens[i]] = lista_pesos[i]

In [10]:
print(pesos)

{'Item_1': 53, 'Item_2': 89, 'Item_3': 221, 'Item_4': 61, 'Item_5': 73, 'Item_6': 241, 'Item_7': 153, 'Item_8': 167, 'Item_9': 194, 'Item_10': 127, 'Item_11': 193, 'Item_12': 191, 'Item_13': 79, 'Item_14': 169, 'Item_15': 97, 'Item_16': 55, 'Item_17': 225, 'Item_18': 142, 'Item_19': 150, 'Item_20': 90}


In [11]:
#Modelagem
m=gp.Model()

#Adiciona as Variaveis de decisão
alocacoes = m.addVars(itens, caixas, vtype = gp.GRB.BINARY)
caixas_usadas = m.addVars(caixas, vtype = gp.GRB.BINARY)

#Restrições - Capacidade das Caixas
c_capacidade = m.addConstrs(gp.quicksum(pesos[i]*alocacoes[i, j] for i in itens)<= capacidade*caixas_usadas[j] for j in caixas)

#Restrição - Cada item tem que estar em uma caixa
c_alocacao = m.addConstrs(gp.quicksum(alocacoes[i, j] for j in caixas) == 1 for i in itens )

#Funcao Objetivo
m.setObjective(gp.quicksum (caixas_usadas[j] for j in caixas), sense = gp.GRB.MINIMIZE)

m.optimize()


Academic license - for non-commercial use only - expires 2021-07-17
Using license file C:\Users\Usuario\gurobi.lic
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 28 rows, 168 columns and 328 nonzeros
Model fingerprint: 0x5817cedb
Variable types: 0 continuous, 168 integer (168 binary)
Coefficient statistics:
  Matrix range     [1e+00, 4e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 8.0000000
Presolve time: 0.03s
Presolved: 28 rows, 168 columns, 328 nonzeros
Variable types: 0 continuous, 168 integer (168 binary)

Root relaxation: objective 6.942500e+00, 49 iterations, 0.01 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    6.94250    0    4    8.00000    6.94250  13

In [12]:
m.objVal

7.0

In [13]:
#Verificar quais itens estão nas caixas
for caixa in caixas:
    print("{}: ".format(caixa), end = " ")
    for item in itens:
        if round(alocacoes[item, caixa].x) == 1:
            print (item, end = " ")
    print('\n')
       

Caixa_1:  Item_1 Item_7 Item_9 

Caixa_2:  Item_14 Item_17 

Caixa_3:  Item_11 Item_12 

Caixa_4:  Item_5 Item_10 Item_16 Item_18 

Caixa_5:  Item_8 Item_13 Item_19 

Caixa_6:  

Caixa_7:  Item_2 Item_3 Item_20 

Caixa_8:  Item_4 Item_6 Item_15 



In [14]:
for caixa in caixas: 
    cap_usada = 0
    for item in itens:
        cap_usada += alocacoes[item, caixa].x*pesos[item]
    print("{}: {}". format(caixa, cap_usada))

Caixa_1: 400.0
Caixa_2: 394.0
Caixa_3: 384.0
Caixa_4: 397.0
Caixa_5: 396.0
Caixa_6: 0.0
Caixa_7: 400.0
Caixa_8: 399.0


In [15]:
for caixa in caixas:
    print(capacidade - c_capacidade[caixa].Slack)

400.0
394.0
384.0
397.0
396.0
400.0
400.0
399.0
